In [2]:
import json

with open('data/clean/stations.json', encoding='utf-8') as file:
    file_contents = file.read()

with open('data/clean/LineStations.json', encoding='utf-8') as file:
    line_contents = file.read()

data = json.loads(file_contents)
lines = json.loads(line_contents)
graph = []
for i in data:
    for j in i['connections']:
        graph.append([i['name'], j['name'], int(j['cost']), 1, j['fromline']])
node_lines = dict()
for i in data:
    node_lines[i['name']] = i['line']

change_lines_time = dict()
for i in data:
    try:
        change_lines_time[i['name']] = i['walkingTime']
    except KeyError:
        pass

temp = []
temp1 = []
totalcost = 0
for i in graph:
    temp.append(i[0])
    temp1.append(i[1])
nodes = set(temp).union(set(temp1))
print(len(nodes))


25
99


In [9]:
def checkChangeLine(curLine, nextNode, fromLine):
    if curLine[-1] not in node_lines[nextNode]:
        return True
    else:
        return False


def findNextLine(curNode, nextNode, fromLine):
    nextLines = set(node_lines[curNode]).intersection(node_lines[nextNode])
    for i in nextLines:
        print(i)


def findWalkingTime(curNode, toLine):
    try:
        if curNode in change_lines_time:
            for i in change_lines_time[curNode]:
                l = dict(i)
                if l["to"] == toLine:
                    return l['time']
        else:
            return 0
    except Exception:
        return 0


def UCS(graph, costs, open, closed, cur_node, totalcosts, line):
    if cur_node in open:
        open.remove(cur_node)

    closed.add(cur_node)
    for i in graph:
        walktime = 0
        if (i[0] == cur_node and costs[i[0]]+i[2] < costs[i[1]] and i[1] not in closed):
            cur_line = line[i[0]]
            if len(cur_line) == 0:
                cur_line.append(i[4])
                line[i[0]] = cur_line
            open.add(i[1])
            next_line = line[i[0]].copy()
            if (checkChangeLine(line[i[0]], i[1], i[4])):
                findNextLine(i[0], i[1], i[4])
                next_line.append(i[4])
                walktime = findWalkingTime(i[0], i[4])
            line[i[1]] = next_line
            extra_cost = 0
            if (line[i[1]][-1] == "Airport Express"):
                extra_cost = 100
            if walktime is None:
                costs[i[1]] = costs[i[0]]+i[2]+extra_cost
            else: 
                costs[i[1]] = costs[i[0]]+i[2]+extra_cost+walktime
            path[i[1]] = path[i[0]] + ' -> ' + i[1]
            # totalcosts[i[1]] = sum(costs[i[1]])
            totalcosts[i[1]] = costs[i[1]]

    costs[cur_node] = 999999
    small = min(costs, key=costs.get)
    if small not in closed:
        UCS(graph, costs, open, closed, small, totalcosts, line)


costs = dict()
path = dict()
totalcosts = dict()
line = dict()
for i in nodes:
    # costs array contain the distance and the heurisitc of current node
    costs[i] = 999999
    path[i] = ' '
    totalcosts[i] = 999999
    line[i] = []
open = set()
closed = set()
start_node = input("Enter the Start Station: ")
open.add(start_node)
path[start_node] = start_node
costs[start_node] = 0
totalcosts[start_node] = 0
UCS(graph, costs, open, closed, start_node, totalcosts, line)
goal_node = input("Enter the Goal Station: ")
print("Suggested Route:")
print(path[goal_node])
changeTimes = len(line[goal_node])-1
for i in line[goal_node]:
    if i == "Walking":
        changeTimes -= 1
print("Interchange times: ", changeTimes)
print("Suggested Line: ", line[goal_node])
print("Least cost is: ", totalcosts[goal_node])


current line: 
['Tuen Ma Line']
check curline:  ['Tuen Ma Line']  -  Siu Hong  -  Tuen Ma Line
Nextline:  ['Tuen Ma Line']
cost:  4
small:  Siu Hong
current line: 
['Tuen Ma Line']
check curline:  ['Tuen Ma Line']  -  Tin Shui Wai  -  Tuen Ma Line
Nextline:  ['Tuen Ma Line']
cost:  10
small:  Tin Shui Wai
current line: 
['Tuen Ma Line']
check curline:  ['Tuen Ma Line']  -  Long Ping  -  Tuen Ma Line
Nextline:  ['Tuen Ma Line']
cost:  14
small:  Long Ping
current line: 
['Tuen Ma Line']
check curline:  ['Tuen Ma Line']  -  Yuen Long  -  Tuen Ma Line
Nextline:  ['Tuen Ma Line']
cost:  17
small:  Yuen Long
current line: 
['Tuen Ma Line']
check curline:  ['Tuen Ma Line']  -  Kam Sheung Road  -  Tuen Ma Line
Nextline:  ['Tuen Ma Line']
cost:  22
small:  Kam Sheung Road
current line: 
['Tuen Ma Line']
check curline:  ['Tuen Ma Line']  -  Tsuen Wan West  -  Tuen Ma Line
Nextline:  ['Tuen Ma Line']
cost:  29
small:  Tsuen Wan West
current line: 
['Tuen Ma Line']
check curline:  ['Tuen Ma Line'